## Set Up Environment

In [ ]:
# Cell 0

network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [ ]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [ ]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [ ]:
# Cell 3

wallet = KeyWallet.load("keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("keystores/balanced_test.json", key_data)

In [ ]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

In [ ]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

### Send ICX to Balanced test wallet

In [ ]:
# Cell 4
# Only necessary if running locally or on the private tbears server
# for the first time since reinitializing.

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(btest_wallet.get_address())\
    .value(1000000 * ICX)\
    .step_limit(1000000) \
    .nid(NID) \
    .nonce(2) \
    .version(3) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

### Deployed to testnet

In [ ]:
# Cell 5
# The following addresses are those deployed to the testnet.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx2594f06f668fa84804d94c1e5d4b378bfcf23b42'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx00093ced37e5c42b995c4df95fa0ce32290c9a36'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx39779100a6556ab3d161dfc95dab3839ac632c9a'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx6dd23ec8b6dc6f6c3dec784e200c097f842b8716'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxaa4593a3d37c12fe29bc7a9507b0a30593e66f65'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxdd1516deed20d0dca0b7ef8c19d2273c48406e86'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx774ddaf4e885782b56c2ddb1df8d8ce3f9ade09e'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx637d21599c04a4c93b390c7f9efb9e7c3a7fcfbb'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx61a36e5d10412e03c907a507d1e8c6c3856d9964'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx2e94568da72a0e4caf58d2baa1366476f05955ef'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx04afcd84c96ee03b30f6f88a60b2a1dee76c63fd'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx53098409b75594a73fb99badff2b34050a3090b6'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx82c8014c8eed1db391109fee2dd0ba11998cd3d2'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx66e56c4ed8a0e7da280867707dcd1ba9322d8c60'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx31bd70011e8ce26cb397ec017a339aabc7e7d049'}}

In [ ]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa0f715fb2c4bc8f4c6399c2cc26167a27be0aa61'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cxbabed822d59b605dbeb6322735c529b292baac3b'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx27aa3bf62145822e60d85fa5d18dabdcff5b9ada'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx01eee12b6614e5328e0a84261652cb7f055e0176'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxd7b3e71dcff3d75392216e208f28ef68e8a54ec0'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx7171e2f5653c1b9c000e24228276b8d24e84f10d'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx799f724e02560a762b5f2bd3b6d2d8d59d7aecc1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [ ]:
{"loans": "cx2594f06f668fa84804d94c1e5d4b378bfcf23b42",
 "staking": "cx00093ced37e5c42b995c4df95fa0ce32290c9a36",
 "dividends": "cx39779100a6556ab3d161dfc95dab3839ac632c9a",
 "reserve": "cx6dd23ec8b6dc6f6c3dec784e200c097f842b8716",
 "daofund": "cxaa4593a3d37c12fe29bc7a9507b0a30593e66f65",
 "rewards": "cxdd1516deed20d0dca0b7ef8c19d2273c48406e86",
 "dex": "cx774ddaf4e885782b56c2ddb1df8d8ce3f9ade09e",
 "governance": "cx637d21599c04a4c93b390c7f9efb9e7c3a7fcfbb",
 "oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964",
 "sicx": "cx2e94568da72a0e4caf58d2baa1366476f05955ef",
 "bnUSD": "cx04afcd84c96ee03b30f6f88a60b2a1dee76c63fd",
 "bnXLM": "cx53098409b75594a73fb99badff2b34050a3090b6",
 "bnDOGE": "cx82c8014c8eed1db391109fee2dd0ba11998cd3d2",
 "baln": "cx66e56c4ed8a0e7da280867707dcd1ba9322d8c60",
 "bwt": "cx31bd70011e8ce26cb397ec017a339aabc7e7d049"}

In [ ]:
for contract in contracts:
    contracts[contract]['SCORE'] = {"loans": "cxfda2a8c77bd8fb44ac07884205e0f920edbff518",
 "staking": "cx56800918000d3ac69b72aa37013fccce0fb4efb8",
 "dividends": "cx6b8c84e69b6286c869ae21992e7394a415e6c48b",
 "reserve": "cx24f70164e1de9b03d7827cd742c0372b5d0faf1d",
 "daofund": "cx871d9c96e31cab6aca4c1468816ad330ef264920",
 "rewards": "cx97130cbc9d0f49d64298c50bc40af5a3bffa1800",
 "dex": "cxc295ccc7606007bfecde9917e0c27459301f9962",
 "governance": "cx805cc8a62e5836b4c8adff6fde2cf18d4285633b",
 "oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964",
 "sicx": "cx299655d0586811ff6173c21946206894c1008ca3",
 "bnUSD": "cxb871c2c0bb8e37df7686c033b47c1e552018794a",
 "bnXLM": "cx888516f7fed8e0d4ea47b1fa60244d820924aa4d",
 "bnDOGE": "cx6ab54e60652a7cbbed8e7358016ac763267f44ae",
 "baln": "cx6420155143c7eac8fa79fee9b5282c9b267ef31c",
 "bwt": "cx5a5e2e315b908b57a88a7818d78410d4fb3bc492"}[contract]

## Deploy All SCOREs

In [ ]:
# Cell 6
# Define deploy and send_tx functions

def compress():
    """
    Compress all SCORE folders in the core_contracts and toekn_contracts folders
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 4000100000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res.get('scoreAddress', '')

def send_tx(dest, value, method, params, wallet):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Calling {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet):
    """
    Compress, Deploy and Configure all SCOREs
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')

    governance = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    contracts['governance']['SCORE'] = governance
    for score in deploy:
        contracts[score]['SCORE'] = deploy_SCORE(contracts[score], {'_governance': governance}, wallet, 0)
    contracts['staking']['SCORE'] = deploy_SCORE(contracts['staking'], {}, wallet, 0)
    contracts['sicx']['SCORE'] = deploy_SCORE(contracts['sicx'], {'_admin': contracts['staking']['SCORE']}, wallet, 0)

    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}

    txns = [{'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}},
            {'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}},
            {'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}}]

    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], wallet)
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}):
    """
    dest is the name of the destination contract.
    """
    print('------------------------------------------------------------------------------------------------------------------')
    print(f'Reading {method}, with parameters {params} on the {dest} contract.')
    print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    print(result)
    return result


In [ ]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

results = {}
deploy_all(btest_wallet)
print('------------------------------------------------------------------------------------------------------------------')
print(contracts)
print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
print(get_scores_json(contracts))

## Deploy or Update a single SCORE

In [ ]:
# Cell 8
# Deploy or Update a single SCORE

compress()
update = 1
contract = contracts['loans']
params = {}
# params = {'_governance': contracts['governance']['SCORE']}
deploy_SCORE(contract, params, btest_wallet, update)


## Test of Typical Flow

In [ ]:
# Cell 9
# 1. staking.stakeICX() - Get 800 ICX worth of sICX from Staking
# 2. loans.addCollateral() - Deposit 800 ICX and Mint 50 bnUSD Loan
# 3. loans.addCollateral() - Just deposit 30 ICX collateral
# 4. loans.originateLoan() - Originate Loan of 10 bnUSD
# 5. loans._repay_loan() via bnUSD.transfer() - Repay 5 bnUSD
# 6. loans.withdrawCollateral() - Withdraw 25 sICX Collateral
# 7. _deposit_and_borrow() via sICX.transfer() - Add 200 sICX collateral directly
# Print results1 if there are any failures.

txns = [{'contract': 'staking', 'value': 800*ICX, 'method': 'stakeICX', 'params': {}},
        {'contract': 'loans', 'value': 800*ICX, 'method': 'addCollateral', 'params': {'_asset': 'bnUSD', '_amount': 50 * ICX}},
        {'contract': 'loans', 'value': 30*ICX, 'method': 'addCollateral', 'params': {'_asset': '', '_amount': 0}},
        {'contract': 'loans', 'value': 0, 'method': 'originateLoan', 'params': {'_asset': 'bnUSD', '_amount': 10 * ICX, '_from': wallet.get_address()}},
        {'contract': 'bnUSD', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['loans']['SCORE'], '_value': 5 * ICX, '_data': json.dumps({"method": "_repay_loan", "params": {}}).encode()}},
        {'contract': 'loans', 'value': 0, 'method': 'withdrawCollateral', 'params': {'_value': 25 * ICX}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['loans']['SCORE'], '_value': 200 * ICX, '_data': json.dumps({"method": "_deposit_and_borrow", "params": {"_sender": wallet.get_address(), "_asset": "", "_amount": 0}}).encode()}}]

results1 = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], btest_wallet)
    results1[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

In [ ]:
# Originate Loan - Without addition of collateral.

params = {"_asset": "bnUSD", "_amount": 50 * ICX, "_from": wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method('originateLoan')\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'************************************************** Status: {res["status"]} **************************************************')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')



### Add new assets

In [ ]:
# Not totally working yet since the Oracle needs to be set through the governance SCORE.
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnDOGE']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('governance', 0, 'addAsset', {'_token_address': contracts['bnXLM']['SCORE'], '_active': True, '_collateral': False}, btest_wallet)
# send_tx('bnDOGE', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)
# send_tx('bnXLM', 0, 'setOracle', {'_address': contracts['oracle']['SCORE']}, btest_wallet)


In [ ]:
call_tx('loans', 'getDebts', {'_address_list': [btest_wallet.get_address(), wallet.get_address()], '_day': 2})


### Test Liquidation

In [ ]:
# Cell 10
# 1. Deposit collateral to fresh wallet
# 2. Check the account standing after depositing collateral - Should be 'No Debt', and
#    holding '0x2a6f1a22364bbe8000' worth of sICX.
# 3a. Mints 200 bnUSD to the test address without checking collateralization ratio
#     With the above collateral deposit it will put the position in a standing of Liquidation.
# 4. Check the new standing of the account. Should have added '0xad78ebc5ac6200000' bnUSD and have standing of 'Liquidate'.

send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
send_tx('loans', 182769 * ICX // 1000, 'addCollateral', {'_asset': '', '_amount': 0}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 600, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 250 * ICX}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)


In [ ]:
# Cell 11
# 4a. If the account does not yet have a standing of 'Liquidate' add another loan of '0xad78ebc5ac6200000'.

send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
send_tx('loans', 1, 'create_test_position', {'_address': btest_wallet.get_address(), '_asset': 'bnUSD', '_amount': 200 * ICX}, btest_wallet)
send_tx('loans', 0, 'toggleTestMode', {}, btest_wallet)
print(call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()}))


In [ ]:
# Cell 12
# 5. Check the account standing after depositing collateral - Should be 'No Debt'.
# 6. Liquidate the account position.
# 7. Check the standing of the account after liquidation. Should now have zero balance for sICX and bnUSD.
# 8. Checking the debts should show the sum of the borrower debt and the bad debt equals the total supply.

call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})
send_tx('loans', 0, 'liquidate', {'_owner': btest_wallet.get_address()}, btest_wallet)
call_tx('loans', 'getAccountPositions', {'_owner': btest_wallet.get_address()})


## Retire some debt to get bnUSD out of Dead Market state

In [ ]:
# Cell 13
# 1. Check that there is a Dead Market
# 2. Check the bad debt, liquidation pool and dead_market state of the asset.
# 3. Retire enough of the bad debt to get the asset out of the dead_market state.
# 4. Again, check the bad debt, liquidation pool and dead_market state of the asset.

call_tx('loans', 'checkDeadMarkets', {})
call_tx('loans', 'getAvailableAssets', {})
send_tx('bnUSD', 0, 'transfer', {'_to': contracts['loans']['SCORE'], '_value': 1000 * ICX, '_data': json.dumps({"method": "_retire_asset", "params": {}}).encode()}, btest_wallet)
call_tx('loans', 'getAvailableAssets', {})


### Unstake some sICX

In [ ]:
send_tx('sICX', 0, 'transfer', {'_to': 'cx135987257c61b3c06cdc0e7b05e3fd50bc68ddc4', '_value': 50 * ICX, '_data': json.dumps({"method": "unstake", "params": {}}).encode()}, btest_wallet)


## Tests with multiple wallets and redemptions

In [ ]:
# Create a set of wallets

# wallets = []
# for i in range(200):
#     wallets.append(KeyWallet.create())
# with open("../wallets.pkl", 'wb') as f:
#     pkl.dump(wallets, f)

In [ ]:
with open("../wallets.pkl", 'rb') as f:
    wallets = pkl.load(f)

In [ ]:
for i in range(20):
    print(f'{wallets[i].get_address()}')

In [ ]:
for i in range(20):
    print(f'{i:3d}: {icon_service.get_balance(wallets[i].get_address()) / 10**18: 9.3f}')

In [ ]:
for i in range(20):
    call_tx("bnUSD", "balanceOf", {"_owner": wallets[i].get_address()})

In [ ]:
# 10000 ICX into each wallet. Uncomment if needed. These wallets are already more than well stocked enough on the tbears server.

# for i in range(10):
#     transaction = TransactionBuilder()\
#         .from_(wallet.get_address())\
#         .to(wallets[i].get_address())\
#         .value(20 * ICX)\
#         .step_limit(1000000) \
#         .nid(NID) \
#         .nonce(2) \
#         .version(3) \
#         .build()
#     signed_transaction = SignedTransaction(transaction, wallet)
#     tx_hash = icon_service.send_transaction(signed_transaction)
# tx_hash

In [ ]:
# Mint some bnUSD to first 20 wallets.

for i in range(20):
    params = {'_asset': 'bnUSD', '_amount': 110 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(800 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# redeem 100 bnUSD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(20):
    params = {'_to': wallets[i+100].get_address(), '_value': 20 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    params = {"method": "_retire_asset", "params": {}}
    data = json.dumps(params).encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 20 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+100].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+100])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Gets the total number of borrowers in the system and iterates over them by index to update each one.

params = {}
call1 = CallBuilder().from_(wallet.get_address())\
                     .to(contracts['loans']['SCORE'])\
                     .method("borrowerCount")\
                     .params(params)\
                     .build()
total_borrowers = int(icon_service.call(call1), 0)
for i in range(total_borrowers):
    params = {'_index': i + 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Mint some bnUSD to each wallet

for i in range(80, 110):
    params = {'_asset': 'bnUSD', '_amount': 100 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(1200 * ICX)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("addCollateral")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# redeem 50 bnUSD from each of 20 wallets that do not have positions on Balanced. This will use up all
# of the liquidation pool and require some replay events to be recorded.

for i in range(80, 110):
    params = {'_to': wallets[i+50].get_address(), '_value': 50 * ICX}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)
    params = {"method": "_retire_asset", "params": {}}
    data = json.dumps(params).encode("utf-8")
    params = {'_to': contracts['loans']['SCORE'], '_value': 50 * ICX, '_data': data}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i+50].get_address())\
        .to(contracts['bnUSD']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i+50])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(1)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for _ in range(10):
    params = {'_index': 1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionAddress")\
                        .params(params)\
                        .build()
    params = {'_owner': icon_service.call(call)}
    transaction = CallTransactionBuilder()\
        .from_(wallets[1].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[1])
    tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(100):
    params = {'_owner': wallets[i].get_address()}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['loans']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("updateStanding")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_holders': [wallet.get_address(), btest_wallet.get_address()]}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
i = 0
{i: [key, result[key]] for i, key in enumerate(result.keys())}

In [ ]:
# Check BALN that has been distributed on the rewards SCORE, and to platform contract addresses.

params = {}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("borrowerCount")\
                    .params(params)\
                    .build()
borrowerCount = int(icon_service.call(call), 0)

addresses = []
for i in range(1, borrowerCount + 1):
    params = {'_index': i, '_day': -1}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['loans']['SCORE'])\
                        .method("getPositionByIndex")\
                        .params(params)\
                        .build()
    position = icon_service.call(call)
    addresses.append(position['address'])

params = {'_holders': addresses}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['rewards']['SCORE'])\
                    .method("getBalnHoldings")\
                    .params(params)\
                    .build()
holders = icon_service.call(call)

total_balances = 0
baln_balances = {}
for contract in ['rewards', 'reserve', 'bwt', 'dex', 'daofund']:
    params = {'_owner': contracts[contract]['SCORE']}
    call = CallBuilder().from_(wallet.get_address())\
                        .to(contracts['baln']['SCORE'])\
                        .method("balanceOf")\
                        .params(params)\
                        .build()
    result = int(icon_service.call(call), 0)
    baln_balances[contract] = result / 10**18
    total_balances += result

i = 0
holdings = {i: [key, int(holders[key], 0), int(holders[key], 0) / 10**18] for i, key in enumerate(holders.keys())}
total = 0
for key in holdings:
    total += holdings[key][1]
    print(f'{holdings[key]}')

print(f'Total unclaimed: {total / 10**18}')
print(baln_balances)
print(f'Total BALN: {total_balances / 10**18}')

In [ ]:
params = {}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['rewards']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("claimRewards")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
sleep(2)

print(i)
res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
for i in range(0, 200):
    params = {}
    transaction = CallTransactionBuilder()\
        .from_(wallets[i].get_address())\
        .to(contracts['rewards']['SCORE'])\
        .value(0)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method("claimRewards")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallets[i])
    tx_hash = icon_service.send_transaction(signed_transaction)
    sleep(2)

    print(i)
    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')

In [ ]:
params = {'_name': 'BALNbnUSD', '_snapshot_id': 1, '_limit': 10, '_offset': 0}
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['loans']['SCORE'])\
                    .method("getDataBatch")\
                    .params(params)\
                    .build()
result = icon_service.call(call)
result

In [ ]:
params = {'_owner': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['loans']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("updateStanding")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
icon_service.get_balance(wallet.get_address()) / 10**18

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('availableBalanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wallet in wallets[:20]:
    address = wallet.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['sicx']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = result

In [ ]:
balances = {}
for wall in [wallet, btest_wallet]:
    address = wall.get_address()
    params = {'_owner': address}
    call = CallBuilder().from_(address)\
                        .to(contracts['baln']['SCORE'])\
                        .method('balanceOf')\
                        .params(params)\
                        .build()
    result = icon_service.call(call)
    balances[address] = int(result, 0) // 10**18
print(balances)

In [ ]:
send_tx('baln', 0, 'transfer', {'_to': 'hx7e02f538732a93893cb225de3e1147a1611ef0ea', '_value': 10**23}, btest_wallet)

### DEX SICX Test

Does the following:

1. Sends ICX to the DEX
2. Checks overall ICX balance
3. Checks withdrawal lock
4. Mints some SICX
5. Trades SICX against ICX

In [ ]:
contracts['dex']['SCORE']

In [ ]:
# Cell 26

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 27

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 28

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXWithdrawLock')\
                    .build()
result = icon_service.call(call)
datetime.utcfromtimestamp(int(result, 0) // 10**6).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# Cell 29

params = {'_to': wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(10 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 30

data = "{\"method\": \"_swap_icx\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(30000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 31

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getICXBalance')\
                    .params({'_address': wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 32

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 33

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDataBatch')\
                    .params({'_name': 'SICXICX', '_limit': 10, '_snapshot_id': 1}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 34

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 10 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
wallet.get_address()

In [ ]:
# Cell 35

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 5 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 36

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['baln']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 37

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getDeposit')\
                    .params({'_user': wallet.get_address(), '_tokenAddress': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 38a

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_maxBaseValue': 100 * ICX, '_quoteValue': 30990197728645406}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 38b

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken': contracts['sicx']['SCORE'], '_quoteToken': contracts['bnUSD']['SCORE'], '_maxBaseValue': 10 * ICX, '_quoteValue': 5 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 39

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("dexPermit")\
    .params({'_pid': 1, '_permission': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 40

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getInversePrice')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**10

In [ ]:
# Cell 41

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 42

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolId')\
                    .params({'_token1Address': contracts['sicx']['SCORE'], '_token2Address': contracts['bnUSD']['SCORE'] }) \
                    .build()
result = icon_service.call(call)
print("pool w/ tokens: " + str(contracts['sicx']['SCORE']) + ' & ' + str(contracts['bnUSD']['SCORE']))
print(result)

In [ ]:
# Cell 43

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolBase')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolQuote')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':2, '_name': 'SICXbnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 44

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNbnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
# Cell 45

to_token = contracts['sicx']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":10}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 45

to_token = contracts['bnUSD']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': ICX // 100, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 46

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['sicx']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 47

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 48

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('totalSupply')\
                    .params({'_pid': 2}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 49

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("remove")\
    .params({'_pid':2, '_value': 1 * ICX, '_withdraw': 1}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolTotal')\
                    .params({'_pid': 2, '_token': contracts['bnUSD']['SCORE']}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [ ]:
# Cell 50

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 30 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 51

transaction = TransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(1 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(101)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
# Cell 52

transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("cancelSicxicxOrder")\
    .params({}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [ ]:
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_pid':3, '_name': 'BALNbnUSD'}) \
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

In [ ]:
to_token = contracts['baln']['SCORE']
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (1 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['bnUSD']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')